In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import ast

from multiprocessing import Pool
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Se llaman los datos necesarios
## Se importan las secuencias de test leaderboard

In [3]:
secuencias_test_leaderboard_df = pd.read_csv('../../Data/secuencias_test_leaderboard_df.csv')
secuencias_test_leaderboard_df

,session_id,secuencia_items_ids,secuencia_fechas
0,26,[19185],['2021-06-16 09:53:54.158']
1,200,"[17089, 17089, 8060, 4758]","['2021-06-25 12:23:40.811', '2021-06-25 12:24:..."
2,205,[8194],['2021-06-11 00:28:07.058']
3,495,[6853],['2021-06-14 22:13:06.741']
4,521,[26471],['2021-06-19 13:50:03.090']
...,...,...,...
49995,4439446,"[20770, 20770, 21396]","['2021-06-11 17:16:12.243', '2021-06-11 17:16:..."
49996,4439458,"[11715, 26499]","['2021-06-09 04:52:02.785', '2021-06-09 04:54:..."
49997,4439550,[19086],['2021-06-02 17:42:40.481']
49998,4439653,"[4813, 14075, 14075, 4813, 12179, 2769, 16479,...","['2021-06-11 10:19:35.472', '2021-06-11 10:19:..."


## Se llaman los items en espacio latente

In [4]:
encoded_items_df = pd.read_csv('../../Data/items_item2vec_encoded.csv')
encoded_items_df

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,2,0.000000e+00,0.000000e+00,8.259485e-38,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,1.147871e-04,1.196613e-25,0.000000,0.000000e+00,4.975298e-21,0.000000,1.000000e+00,1.283712e-37,0.000000e+00,5.708601e-11
1,3,7.226150e-16,2.417073e-01,2.325430e-14,3.434116e-10,9.999780e-01,2.084914e-19,2.359839e-04,7.360519e-18,3.367935e-12,...,6.631301e-28,1.442925e-15,1.000000,7.131817e-04,9.999999e-01,0.034609,8.714515e-01,5.728165e-07,1.033759e-04,9.978011e-01
2,4,7.434464e-30,7.750213e-18,0.000000e+00,1.445317e-29,1.000000e+00,0.000000e+00,0.000000e+00,3.267208e-25,9.452282e-33,...,0.000000e+00,5.438013e-28,1.000000,5.429324e-03,1.000000e+00,1.000000,9.924654e-01,6.889178e-05,9.211239e-23,2.146675e-14
3,7,5.490076e-37,0.000000e+00,1.191505e-21,9.322182e-12,4.260253e-13,5.699080e-01,1.000000e+00,4.086731e-30,5.050279e-12,...,1.000000e+00,0.000000e+00,0.000000,1.472102e-12,4.147344e-34,0.000000,0.000000e+00,1.565254e-36,1.000000e+00,1.000000e+00
4,8,2.423661e-25,2.795917e-11,6.869838e-35,1.957358e-24,1.000000e+00,1.214310e-37,0.000000e+00,2.739703e-17,1.734631e-19,...,0.000000e+00,1.350020e-19,1.000000,7.157927e-10,1.000000e+00,1.000000,2.604076e-02,1.155714e-02,3.667003e-09,1.811880e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23686,28139,9.726211e-01,2.284531e-11,2.776817e-11,5.597994e-02,1.026659e-02,2.461190e-02,1.000000e+00,3.444716e-18,1.398369e-05,...,4.171538e-03,1.981965e-04,0.961271,7.362516e-17,3.345782e-01,0.000326,3.236383e-25,6.360826e-16,9.999509e-01,9.999608e-01
23687,28140,9.997767e-01,2.538743e-08,7.991548e-11,2.896282e-21,3.695540e-04,9.091385e-18,2.061393e-09,1.558076e-14,5.036157e-05,...,3.120203e-29,1.000000e+00,1.000000,2.877177e-09,2.796884e-04,0.615862,9.811885e-01,1.042302e-14,2.447064e-09,9.952277e-01
23688,28141,2.941844e-15,7.376524e-11,3.965195e-17,8.104620e-05,9.995428e-01,5.364074e-11,4.411794e-07,4.666649e-08,1.239170e-11,...,2.487879e-37,3.125461e-19,1.000000,4.307819e-08,1.000000e+00,0.987323,9.997515e-01,3.231751e-02,1.217761e-02,8.776583e-06
23689,28142,0.000000e+00,1.112993e-36,1.000000e+00,1.000000e+00,3.748123e-22,1.000000e+00,1.000000e+00,1.000000e+00,1.470198e-26,...,1.000000e+00,0.000000e+00,0.000000,5.781284e-18,0.000000e+00,0.000000,6.874122e-17,0.000000e+00,1.000000e+00,5.961677e-13


In [5]:
candidate_items_df = pd.read_csv('../../Data/dressipi_recsys2022/candidate_items.csv')
candidate_items_df

,item_id
0,4
1,8
2,9
3,19
4,20
...,...
4985,28128
4986,28131
4987,28132
4988,28133


In [6]:
candidate_items_encoded_df = pd.merge(candidate_items_df,encoded_items_df,on='item_id')
candidate_items_encoded_df

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,4,7.434464e-30,7.750213e-18,0.000000e+00,1.445317e-29,1.000000,0.000000e+00,0.000000e+00,3.267208e-25,9.452282e-33,...,0.000000e+00,5.438013e-28,1.000000,5.429324e-03,1.000000,1.000000,9.924654e-01,6.889178e-05,9.211239e-23,2.146675e-14
1,8,2.423661e-25,2.795917e-11,6.869838e-35,1.957358e-24,1.000000,1.214310e-37,0.000000e+00,2.739703e-17,1.734631e-19,...,0.000000e+00,1.350020e-19,1.000000,7.157927e-10,1.000000,1.000000,2.604076e-02,1.155714e-02,3.667003e-09,1.811880e-13
2,9,1.308194e-28,6.645637e-27,0.000000e+00,2.939266e-21,1.000000,0.000000e+00,0.000000e+00,3.075361e-08,9.543768e-18,...,0.000000e+00,9.999991e-01,1.000000,2.129090e-13,1.000000,1.000000,1.450176e-12,1.000000e+00,9.866864e-20,7.057842e-08
3,19,0.000000e+00,1.927579e-24,0.000000e+00,1.369518e-34,1.000000,0.000000e+00,0.000000e+00,2.840138e-17,1.526441e-38,...,0.000000e+00,2.147005e-27,1.000000,6.772441e-06,1.000000,1.000000,1.000000e+00,1.233657e-25,1.387237e-20,2.142658e-22
4,20,5.769076e-11,3.288506e-04,9.161456e-09,2.319106e-09,0.999709,2.632268e-15,2.761055e-02,3.668244e-11,8.201424e-10,...,4.000478e-22,3.423259e-11,0.999808,1.307118e-04,0.999819,0.002696,9.858159e-01,5.706286e-06,2.519414e-05,9.900947e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,28128,4.469821e-31,4.933021e-29,0.000000e+00,1.461128e-20,1.000000,0.000000e+00,0.000000e+00,3.827549e-14,1.402932e-01,...,0.000000e+00,9.537469e-26,1.000000,2.898941e-13,1.000000,1.000000,4.324041e-17,1.000000e+00,9.698453e-01,2.418449e-20
4986,28131,2.254948e-27,2.096988e-02,2.682238e-24,4.560917e-32,0.999998,0.000000e+00,0.000000e+00,1.916292e-35,6.390052e-18,...,0.000000e+00,9.554233e-12,1.000000,2.539598e-01,1.000000,1.000000,1.002942e-15,5.579268e-25,2.983091e-02,1.358837e-15
4987,28132,1.000000e+00,2.885962e-11,0.000000e+00,1.874853e-27,0.000003,0.000000e+00,0.000000e+00,8.162775e-27,9.999996e-01,...,0.000000e+00,1.000000e+00,1.000000,3.023659e-16,0.988429,1.000000,1.217375e-16,7.095374e-21,1.872294e-12,9.199441e-01
4988,28133,1.387520e-05,6.941578e-05,9.354117e-01,9.999938e-01,0.411254,1.000000e+00,1.000000e+00,6.676278e-02,7.858561e-03,...,5.395652e-01,5.330301e-06,0.011493,3.583865e-03,1.000000,0.995892,9.999293e-01,5.873812e-08,9.963646e-01,1.000000e+00


# 1. Se convierten las secuencias en tensores para alimentar la red recurrente

In [6]:
def normalize_len_sequence(sequence_list, len_output = 10):
    
    if len(sequence_list) >= len_output:
        
        output_list = sequence_list[-1*len_output:]
        
    else:
        
        diference_len = len_output - len(sequence_list)
        padding = [0] * diference_len 
        output_list = padding + sequence_list
        
    return output_list


def create_tensors_for_rnn_for_testing(secuencias_df, items_df, length_sequence = 10):
    
    X = []
    Y = []
    
    example_tensor = items_df.loc[[0]].drop('item_id', axis=1).to_numpy()
    
    for index, row in tqdm(secuencias_df.iterrows()):
        
        secuencia_items = ast.literal_eval(row.secuencia_items_ids)
        secuencia_items_normalizada = normalize_len_sequence(secuencia_items, length_sequence)
        
        secuencia_fechas_items = ast.literal_eval(row.secuencia_fechas)
        secuencia_fechas_items_normalizada = normalize_len_sequence(secuencia_fechas_items, length_sequence)
        
        X_secuencia_individual = []
        
        for item in secuencia_items_normalizada:
                        
            if item == 0:
                
                item_data_tensor = np.zeros(example_tensor.shape)
                X_secuencia_individual.append(item_data_tensor)
                
            else:
                
                item_data_tensor = items_df.loc[items_df.item_id == item].drop('item_id', axis=1).to_numpy()
                X_secuencia_individual.append(item_data_tensor)
        
        
        X_secuencia_individual = np.array(X_secuencia_individual)
        X_secuencia_individual = np.reshape(X_secuencia_individual, (X_secuencia_individual.shape[0], X_secuencia_individual.shape[2]))
        
        X.append(X_secuencia_individual)
        

    X = np.array(X)
    
    return X

In [8]:
X_10_leaderboard = create_tensors_for_rnn_for_testing(secuencias_test_leaderboard_df, encoded_items_df, 10)

50000it [01:16, 655.34it/s]


In [9]:
print(X_10_leaderboard.shape)

(50000, 10, 64)


# 2. Se procesan las secuencias en el modelo RNN

In [7]:
model_bilstm = tf.keras.models.load_model('../../Models/biLSTM_v1')
model_bilstm.summary()

Model: "model_bilstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Inputs (InputLayer)          [(None, 10, 64)]          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 32)            10368     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32)                6272      
_________________________________________________________________
Output (Dense)               (None, 64)                2112      
Total params: 18,752
Trainable params: 18,752
Non-trainable params: 0
_________________________________________________________________


In [10]:
predicted_latent_items_leaderboard = model_bilstm(np.float32(X_10_leaderboard[0:1,:,:])).numpy()

for i in tqdm(range(1, X_10_leaderboard.shape[0])):
    
    predicted_item = model_bilstm(np.float32(X_10_leaderboard[i:i+1,:,:])).numpy() 
    
    predicted_latent_items_leaderboard = np.vstack((predicted_latent_items_leaderboard,predicted_item))

100%|███████████████████████████████████████████████████████████████████████████| 49999/49999 [04:27<00:00, 187.02it/s]


In [11]:
predicted_latent_items_leaderboard.shape

(50000, 64)

In [12]:
np.save('../../Data/predicted_latent_items_leaderboard', predicted_latent_items_leaderboard)

In [7]:
predicted_latent_items_leaderboard = np.load('../../Data/predicted_latent_items_leaderboard.npy')
predicted_latent_items_leaderboard.shape

(50000, 64)

# 3. Generacion de los items de salida empleando la similitud del coseno

In [8]:
sessions_ids = list(secuencias_test_leaderboard_df['session_id'])
print(sessions_ids[-5:])
print(len(sessions_ids))

[4439446, 4439458, 4439550, 4439653, 4439757]
50000


In [40]:
def get_rank_of_prediction(cosine_similarity_list, items_ids_list, top = 100):
    
    cosine_similarity_items_ids = pd.DataFrame(
    {
        'item_id': items_ids_list,
        'cosine_similarity': cosine_similarity_list
    })
    
    cosine_similarity_items_ids.sort_values(by='cosine_similarity',ascending=False, inplace=True)
        
    items_ids_top = list(cosine_similarity_items_ids['item_id'])[:top]
    
    
    return items_ids_top



def generate_submission(lista_ids_sesiones, predicted_latent_items, items_encoded):
    
    submission_df = pd.DataFrame(columns=['session_id','item_id','rank'])
    
    encoded_items_tensor = items_encoded.drop('item_id', axis=1).to_numpy()
    ids_items_encoded_list = list(items_encoded['item_id'])
    
    cosine_similarity_results = cosine_similarity(predicted_latent_items, encoded_items_tensor)
    
    for i in tqdm(range(len(lista_ids_sesiones))):
        
                
        cosine_similarity_result_list = list(cosine_similarity_results[i])
        
        top_items_ids = get_rank_of_prediction(cosine_similarity_result_list, ids_items_encoded_list, top = 100)
        
        session_list = [lista_ids_sesiones[i]] * 100
        ranks_list = list(range(1, 101))
        
        temp_df = pd.DataFrame(
            {'session_id': session_list,
             'item_id': top_items_ids,
             'rank': ranks_list
            })
        
        submission_df = submission_df.append(temp_df)
    
    return submission_df


In [41]:
submission_leaderboard = generate_submission(sessions_ids, predicted_latent_items_leaderboard, candidate_items_encoded_df)

100%|██████████| 50000/50000 [55:50<00:00, 14.92it/s]


In [42]:
submission_leaderboard

,session_id,item_id,rank
0,26,25815,1
1,26,22378,2
2,26,7644,3
3,26,18502,4
4,26,18418,5
...,...,...,...
95,4439757,19509,96
96,4439757,19663,97
97,4439757,17059,98
98,4439757,19181,99


In [43]:
submission_leaderboard.to_csv('../../Data/submission_leaderboard_item2vec_bilstmv1.csv', index=False)